In [1]:
import os
import sys
from pathlib import Path

mod_path = os.path.join(Path.cwd().parent.parent)
if mod_path not in sys.path:
    sys.path.append(mod_path)

In [2]:
import pandas as pd
import numpy as np
import random
from copy import deepcopy

from typing import List, Tuple
from sklearn.model_selection import StratifiedKFold

from src.features import *
from src.features.utils import *
from src.model.tree_based import ModelXgBoost
from src.data.validation_file import SubmissionFile

np.random.seed(0)
random.seed(0)

In [3]:
# data with shape 28007, 33 [transaction related features]
train = pd.read_csv('../../data/processed/train.csv')
test_set = pd.read_csv('../../data/processed/test.csv')

In [ ]:
# data with shape 28007, 33 [transaction related features]
train = pd.read_csv('../../data/processed/train2.csv')
test_set = pd.read_csv('../../data/processed/test2.csv')

In [4]:
train.drop(columns = 'Unnamed: 0', inplace=True)
test_set.drop(columns = 'Unnamed: 0', inplace=True)
# test_set.columns

In [5]:
train.shape

(28007, 35)

In [6]:
# Region has certain NaN values which might cause issues while encoding
# As total NaNs constitute ~5% of the data (1446) we remove it as of now
print(train['Region'].isna().sum() / train.shape[0] * 100)
train.dropna(subset=['Region'], how='all', inplace=True)


5.162994965544328


In [ ]:
# Region has certain NaN values which might cause issues while encoding
# As total NaNs constitute ~5% of the data (1446) we remove it as of now
print(test_set['Region'].isna().sum() / test_set.shape[0] * 100)
# train.dropna(subset=['Region'], how='all', inplace=True)

# When attempting drop=first in OHE, the reverse transform throws an issue as it reads the NaN values as a separate
# category. So converting NaNs into strings
train['Region'] = train['Region'].fillna('Null')
test_set['Region'] = test_set['Region'].fillna('Null')

## Approach 2

s = pd.DataFrame(np.arange(0, len(train)), columns=['m1'])
df = train[['b1', 'b2', 'b3', 'b4', 'b5']]
df.drop(columns=['b5'], inplace=True)
df.rename(columns={'b1': 'b2', 'b2':'b3', 'b3': 'b4', 'b4': 'b5'}, inplace=True)
df.insert(loc=0, column='b1', value=s.values)
df

t = pd.DataFrame([[1]], columns=['a'])
q = pd.DataFrame([[1]], columns=['b'])
r = pd.DataFrame([['aum']], columns=['name'])

k = pd.concat([t, q])
kk = pd.merge(k, r, how='left', left_index=True, right_index=True)
kk

In [7]:
def slide_variable_window(
    predictor_array: pd.DataFrame,
    var_to_add: pd.DataFrame
) -> pd.DataFrame:
    predictor_array.drop(columns=['b5'], inplace=True)  # We drop the first payment
    predictor_array.rename(columns={'b1': 'b2', 'b2':'b3', 'b3': 'b4', 'b4': 'b5'}, inplace=True)
    predictor_array.insert(loc=0, column='b1', value=var_to_add.values)  # And add the new variable (mn)
    
    return predictor_array

In [8]:
# split_payment_history_df = train[["ID", "SplitPaymentsHistory"]]
# id_arr = train[["ID"]]

target = train[['m1', 'm2', 'm3', 'm4', 'm5', 'm6']]
train_arr = train.drop(columns=['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 
                                'SplitPaymentsHistory',
                                'ExpectedTermDate', 
                                'FirstPaymentDate',
                                'LastPaymentDate'])
test_set.drop(columns=['SplitPaymentsHistory',
                       'ExpectedTermDate', 
                       'FirstPaymentDate',
                       'LastPaymentDate'], inplace=True)

In [9]:
def create_data_with_sliding_approach(data_without_target: pd.DataFrame,
                                      target_data: pd.DataFrame):
    frame = pd.DataFrame(None)
    new_df = deepcopy(data_without_target)
    target_df = pd.DataFrame(None)
    target_features = target_data.columns.tolist()
    for itr, col in enumerate(target_features):
        if itr == 0:
            target_df = pd.concat([target_df, target_data[[col]]])
            frame = pd.concat([frame, data_without_target])
        else:
            filter_df = new_df[['b1', 'b2', 'b3', 'b4', 'b5']]  # Intermediate df
            new_df.drop(columns=['b1', 'b2', 'b3', 'b4', 'b5'], inplace=True)
            concatinating_df = slide_variable_window(predictor_array=filter_df, 
                                                     var_to_add=target_data[[target_features[itr-1]]])
            new_df = pd.concat([new_df, concatinating_df], axis=1)  # We add the newly created columns
            target_df = pd.concat([target_df, target_data[[col]]])
            frame = pd.concat([frame, new_df])
#             print(new_df.shape)

    target_df = pd.DataFrame(target_df.sum(axis=1).astype(int), columns=['target'])
#     print(frame.shape)  # Should be 6 * original data's no. of rows
    
    frame.reset_index(drop=True, inplace=True)
    target_df.reset_index(drop=True, inplace=True)
    
    return frame, target_df

In [10]:
# split data into train and test sets
from sklearn.model_selection import train_test_split

seed = 10
X_train, X_test, y_train, y_test = train_test_split(train_arr, target, test_size=0.45, random_state=seed)

### Model train on initial hp :: Approach 2

def approach_two_model(x_train, y_train, x_test):
    model = ModelXgBoost(train_array=x_train, train_target=y_train)
    model.train_model()  # Default h.params (Checkout the code)
    predict = model.trained_model.predict(x_test)
    
    return model, predict

In [11]:
def encode_and_drop(full_array, data_type, tr_encoder=None):
#     print(tr_encoder)
    categorical_array = full_array[full_array.select_dtypes(exclude=['number']).columns]
    numerical_array = full_array.drop(columns=full_array.select_dtypes(exclude=['number']).columns)
#     print(categorical_array.columns)
    if not tr_encoder:
        encoder = FeatureEncoding()
    else:
        encoder = tr_encoder
    encoded_array = encoder.one_hot_encoding(
        categorical_frame=categorical_array, 
        type_of_data=data_type,
        conv=True,
        drop=None,
        handle_unknown="ignore"
    )
#     print(encoded_array.columns)
    final_array = pd.concat([numerical_array.reset_index(drop=True), 
                             encoded_array.reset_index(drop=True)], axis=1)
    final_array.index = numerical_array.index
#     print(final_array.shape)
    return numerical_array, encoded_array, final_array, encoder

In [ ]:
def get_top_features(
    features,
    feature_scores,
    cut_off_score=0.8
) -> Tuple[List[List], List[List]]:
    frame = pd.DataFrame([feature_scores], columns=features, index=['gain']).T
    frame.sort_values(by=['gain'], ascending=False, inplace=True)
    frame['cum_gain'] = frame['gain'].cumsum()
    
    feature_list = list(frame[frame['cum_gain'] <= cut_off_score].index)
    feature_scores_list = frame[frame['cum_gain'] <= cut_off_score]['gain'].tolist()
    
    return feature_list, feature_scores_list

In [ ]:
def get_important_feature_scores(
    feature_corpus: List[List],
    score_corpus: List[List]
) -> pd.DataFrame:
    imp_column = dict()
    imp_col = dict()
    for feature_list, score_list in zip(feature_corpus, score_corpus):
        for feature_names, scores in zip(feature_list, score_list):
            if feature_names not in imp_column.keys():
                imp_column[feature_names] = 1
#                 imp_col[feature_names] = [scores]
            else:
                imp_column[feature_names] += 1
#                 imp_col[feature_names].append(scores)
    
    important_features_df = pd.DataFrame(imp_column, index=['frequency']).T
#     important_scores = pd.DataFrame(imp_col)
    important_features_df['appearance_ratio'] = important_features_df['frequency'] / 100
    important_features_df.sort_values(by=['frequency'], ascending=False, inplace=True)
#     important_features_df['cum_score'] = important_features_df['score'].cumsum()
    
    return important_features_df

In [ ]:
# Preparing the TRAIN data for approach two and fitting the model
train_data, target_frame = create_data_with_sliding_approach(data_without_target=train_arr, 
                                                             target_data=target)
id_array = train_data[["ID"]]
train_data.drop(columns=["ID"], inplace=True)
num_frame, encoded_train, full_frame, encoder_model = encode_and_drop(train_data, "train", None)

# encoded_train = pd.concat([encoded_train, train_data[['b1', 'b2', 'b3', 'b4', 'b5']]], axis=1)
print('Columns ->', encoded_train.columns)
print('Shape ->', encoded_train.shape)


np.random.seed(0)
r = np.random.randint(1, 100, 1)
f_i_list = []
f_i_features = []
for i, _seed in enumerate(r):
#     sample= full_frame.sample(n=20000, replace=False, random_state=_seed)
#     target_f = target_frame[target_frame.index.isin(sample.index)]
    sample= full_frame
    target_f = target_frame
    model_two_obj = ModelXgBoost(train_array=sample, 
                                 train_target=target_f)
    model_two_obj.train_model()  # Default h.params (Checkout the code)
    model_two = model_two_obj.trained_model
    f_i = model_two.feature_importances_
    f_i_cols = sample.columns
    variables, variable_scores = get_top_features(features=f_i_cols, feature_scores=f_i, cut_off_score=0.8)
    f_i_list.append(variable_scores)
    f_i_features.append(variables)

important_features_df = get_important_feature_scores(feature_corpus=f_i_features, score_corpus=f_i_list)

In [ ]:
important_features_df

In [ ]:
important_features_df

In [ ]:
frame = pd.DataFrame([variable_scores], columns=variables, index=['Gain']).T
frame.sort_values(by=['Gain'], ascending=False, inplace=True)
frame['cumulative_gain'] = frame['Gain'].cumsum()
frame
# frame.to_csv('cumulative_gain_og_features.csv')

In [ ]:
important_features_df.index

In [ ]:
# important_features_df.to_csv('../../submissions/important_features_total_gain_with_80_cutoff.csv')

In [ ]:
from matplotlib import pyplot

pyplot.figure(figsize=(15,10))
pyplot.bar(frame.index, frame['cumulative_gain'])
# xgb.plot_importance(model_two)
pyplot.show()

In [ ]:
# When train has Null regions removed
# feature_selection = list(important_features_df[important_features_df['appearance_ratio'] >=0.3].index)
feature_selection = ['North Rift',
 'Other',
 'Term',
 'Driver/Motorbike Rider',
 'Labourer',
 'South Rift',
 'TotalContractValue',
 'Nyanza',
 'Nairobi Region',
 'Western',
 'Teacher',
 'Mount Kenya Region',
 'Government Employee',
 'Farmer',
 'Male',
 'AccessoryRate',
 'RatePerUnit',
 'WEEKLY']

In [ ]:
# When train has Null regions as a category
# feature_selection = list(important_features_df[important_features_df['appearance_ratio'] <=0.5].index)
feature_selection = ['b1', 'Age', 'b4', 'DaysOnDeposit', 'b2', 'Deposit', 'b3', 'North Rift',
       'b5', 'South Rift', 'Term', 'Other', 'Driver/Motorbike Rider',
       'Coast Region', 'Nairobi Region', 'Western', 'Labourer', 'Teacher', 'Mount Kenya Region', 'Farmer',
       'Nyanza', 'Government Employee', 'Business']
# feature_selection = ['Driver/Motorbike Rider',
# #  'Coast Region',
#  'Nairobi Region',
#  'Western',
#  'Labourer',
#  'TotalContractValue',
#  'Teacher',
#  'Mount Kenya Region',
#  'Farmer',
#  'Nyanza',
#  'Government Employee',
# #  'Female',
# #  'Business',
#  'RatePerUnit',
#  'AccessoryRate',
#  'WEEKLY']

In [25]:
encoded_train.columns

Index(['DAILY', 'MONTHLY', 'WEEKLY', 'Female', 'Male', 'Coast Region',
       'Mount Kenya Region', 'Nairobi Region', 'North Rift', 'Nyanza',
       'South Rift', 'Western', 'Business', 'Driver/Motorbike Rider', 'Farmer',
       'Government Employee', 'Labourer', 'Other', 'Teacher', 'b1', 'b2', 'b3',
       'b4', 'b5'],
      dtype='object')

In [12]:
# Preparing the TRAIN data for approach two and fitting the model
train_data, target_frame = create_data_with_sliding_approach(data_without_target=X_train, 
                                                             target_data=y_train)
id_array = train_data[["ID"]]
train_data.drop(columns=["ID"], inplace=True)
num_frame, encoded_train, full_train, encoder_model = encode_and_drop(train_data, "train", None)
# encoded_train = encoded_train[feature_selection]
print(encoded_train.shape)
encoded_train = pd.concat([encoded_train, train_data[['b1', 'b2', 'b3', 'b4', 'b5']]], axis=1)
print(encoded_train.shape)

model_two_obj = ModelXgBoost(train_array=encoded_train,
                             train_target=target_frame)
model_two_obj.train_model()  # Default h.params (Checkout the code)
model_two = model_two_obj.trained_model


/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


(87648, 19)
(87648, 24)
PARAMS-> {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'gamma': None, 'gpu_id': None, 'importance_type': 'gain', 'interaction_constraints': None, 'learning_rate': None, 'max_delta_step': None, 'max_depth': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 0, 'reg_alpha': 50, 'reg_lambda': 0, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': 0, 'use_label_encoder': False}


### Prediction using Model :: Approach 2

In [13]:
test_id_array = X_test[["ID"]]
X_test.drop(columns=["ID"], inplace=True)

/home/aumaron/.local/share/virtualenvs/zindi_payg-FXkRANRI/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [14]:
X_test.index

Int64Index([19204, 15721,  7879, 25250, 19371,  2608,  2121, 20788, 20931,
            21136,
            ...
            26884,  7697, 19076, 24308,  9246, 22564, 17551, 14648,  3550,
            13387],
           dtype='int64', length=11953)

In [15]:
# Encoding and re-attaching using train encoding model
num_frame_test, encoded_test, full_test, encoder_model = encode_and_drop(X_test, "test", encoder_model)
# encoded_test = encoded_test[feature_selection]
_inexes = X_test.index
encoded_test = pd.concat([encoded_test.reset_index(drop=True),
                          X_test[['b1', 'b2', 'b3', 'b4', 'b5']].reset_index(drop=True)], axis=1)
encoded_test.index = _inexes
print(encoded_test.shape)
predict_dict = dict()
for col_no, predict_col in enumerate(y_test.columns):
#     print(encoded_test)
    predict_dict[f"{predict_col}_pred"] = model_two.predict(encoded_test)
    print(predict_dict[f"{predict_col}_pred"])
    int_df = encoded_test[['b1', 'b2', 'b3', 'b4', 'b5']]
    encoded_test.drop(columns=['b1', 'b2', 'b3', 'b4', 'b5'], inplace=True)
    concatinating_df = slide_variable_window(predictor_array=deepcopy(int_df), 
                                             var_to_add=pd.DataFrame(deepcopy(predict_dict[f"{predict_col}_pred"])))
    encoded_test = pd.concat([encoded_test, concatinating_df], axis=1)  # We add the newly created columns
#     print(encoded_test[['b1', 'b2', 'b3', 'b4', 'b5']])
    
predict_dict

(11953, 24)
[ 298.98547 1370.1808   336.08096 ...  229.55864 1385.3893  1319.7733 ]
[ 365.25754 1408.1921   404.01724 ...  223.63448 1303.6301  1298.9755 ]
[ 321.3584  1430.3431   447.55276 ...  223.63448 1314.6316  1222.1951 ]
[ 379.00632 1279.6415   432.97665 ...  209.94781 1338.343   1241.9469 ]
[ 359.7216  1228.2202   387.7058  ...  263.27692 1250.7705  1142.567  ]
[ 414.9437  1168.9843   365.17502 ...  247.06192 1127.6016  1129.6669 ]


{'m1_pred': array([ 298.98547, 1370.1808 ,  336.08096, ...,  229.55864, 1385.3893 ,
        1319.7733 ], dtype=float32),
 'm2_pred': array([ 365.25754, 1408.1921 ,  404.01724, ...,  223.63448, 1303.6301 ,
        1298.9755 ], dtype=float32),
 'm3_pred': array([ 321.3584 , 1430.3431 ,  447.55276, ...,  223.63448, 1314.6316 ,
        1222.1951 ], dtype=float32),
 'm4_pred': array([ 379.00632, 1279.6415 ,  432.97665, ...,  209.94781, 1338.343  ,
        1241.9469 ], dtype=float32),
 'm5_pred': array([ 359.7216 , 1228.2202 ,  387.7058 , ...,  263.27692, 1250.7705 ,
        1142.567  ], dtype=float32),
 'm6_pred': array([ 414.9437 , 1168.9843 ,  365.17502, ...,  247.06192, 1127.6016 ,
        1129.6669 ], dtype=float32)}

In [16]:
for k, v in predict_dict.items():
#     predict_dict[k] = [0 if i < 0 else i for i in v]
    for _v in v:
        if _v < 0:
            print(_v)
# predict_dict

-392.03134
-336.96024
-256.6005
-116.88847
-7.944236
-269.73184
-673.13513
-15.122854
-172.21228
-324.7183
-4.881765
-853.0861
-60.498684
-20.132063
-196.47621
-492.92197
-115.79759
-10.721448
-11.710172
-11.236606
-8.821763


### Calculation of RMSE

In [17]:
pred_frame = pd.DataFrame(predict_dict)
pred_frame.index = X_test.index

In [18]:
full_test_array = pd.concat([X_test, y_test, pred_frame], axis=1)

In [19]:
full_test_array = pd.merge(full_test_array, test_id_array, how='left', left_index=True, right_index=True)

In [20]:
sub_file = SubmissionFile(
    validation_data=full_test_array,
    type_of_data='validation'
).execute()

In [21]:
sub_file.shape

(71718, 3)

In [22]:
sub_file['SquaredError'] = np.square(sub_file['Target'] - sub_file['Prediction'])

In [23]:
rmse = np.sqrt(np.sum(sub_file['SquaredError'])/sub_file.shape[0])
print('Final RMSE --> ', rmse)

Final RMSE -->  836.3801053455517


### Preparing Submission

In [ ]:
# Preparing the Entire TRAIN data for approach two and fitting the model
train_data, target_frame = create_data_with_sliding_approach(data_without_target=train_arr, 
                                                             target_data=target)
print(train_data.columns)
id_array = train_data[["ID"]]
train_data.drop(columns=["ID"], inplace=True)
og_frame, encoded_train, encoder_model = encode_and_drop(train_data, "train", None)
print(encoded_train.columns)
encoded_train = encoded_train[feature_selection]
print(encoded_train.columns)
encoded_train = pd.concat([encoded_train, train_data[['b1', 'b2', 'b3', 'b4', 'b5']]], axis=1)

model_two_obj = ModelXgBoost(train_array=encoded_train, 
                             train_target=target_frame)
model_two_obj.train_model()  # Default h.params (Checkout the code)
model_two = model_two_obj.trained_model

# Strategy to impute missing regions | did not work
most_common_region = test_set['Region'].value_counts().index[0]
test_set.loc[test_set['Region'].isna(), 'Region'] = most_common_region

In [ ]:
test_id = test_set[["ID"]]
test_set.drop(columns=["ID"], inplace=True)

# Encoding and re-attaching using train encoding model
og_frame_test, encoded_test, encoder_model = encode_and_drop(test_set, "test", encoder_model)
encoded_test = encoded_test[feature_selection]
encoded_test = pd.concat([encoded_test, test_set[['b1', 'b2', 'b3', 'b4', 'b5']].reset_index(drop=True)], axis=1)

predict_dict = dict()

for col_no, predict_col in enumerate(['m1', 'm2', 'm3', 'm4', 'm5', 'm6']):
    predict_dict[f"m{col_no+1}_pred"] = model_two.predict(encoded_test)
    int_df = encoded_test[['b1', 'b2', 'b3', 'b4', 'b5']]
    encoded_test.drop(columns=['b1', 'b2', 'b3', 'b4', 'b5'], inplace=True)
    concatinating_df = slide_variable_window(predictor_array=int_df, 
                                             var_to_add=pd.DataFrame(predict_dict[f"m{col_no+1}_pred"]))
    encoded_test = pd.concat([encoded_test, concatinating_df], axis=1)  # We add the newly created columns
    
# predict_dict

In [ ]:
for ke, va in predict_dict.items():
    predict_dict[ke] = [0 if i < 0 else i for i in va]
    for _va in va:
        if _va < 0:
            print(_va)

In [ ]:
pred_frame_test = pd.DataFrame(predict_dict)
pred_frame_test.index = test_set.index

In [ ]:
full_test_array_test = pd.concat([test_set, pred_frame_test], axis=1)
full_test_array_test = pd.merge(full_test_array_test, test_id, how='left', left_index=True, right_index=True)

In [ ]:
sub_file = SubmissionFile(
    validation_data=full_test_array_test,
    type_of_data='test'
).execute()
sub_file.reset_index(drop=True, inplace=True)

In [ ]:
sub_file.to_csv('../../submissions/submission_approach_2_bottom_feature_selection_null_region_category_neg_predictions_removed.csv')

In [ ]:
sub_file